# CNNs in TensorFlow

In [2]:
k = 64

img_size = [10,10,3]

filter_size = [5,5]

import tensorflow as tf

# image 
input_img = tf.placeholder(tf.float32, shape=[None, *img_size])

# weight + bias
weight = tf.Variable(tf.truncated_normal([*filter_size, img_size[2], k]))
bias = tf.Variable(tf.zeros(k))

# hyperparams
strides = [1,2,2,1]
padding = 'SAME'

# apply convolution 
conv_layer = tf.nn.conv2d(input_img, weight, strides=strides, padding=padding)
# add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
# apply activation
conv_layer = tf.nn.relu(conv_layer)

## TensorFlow Max Pooling 

![max](image/max-pooling.png)

The image above is an example of max pooling with a 2x2 filter and stride of 2. The four 2x2 colors represent each time the filter was applied to find the maximum value.

Benefit of max pooling is to reduce size of the input and allows CNN to focus on only the most important elements (hence retaining only the max value for each filtered area and removing the rest) 

TF provides `tf.nn.max_pool()` function to apply max pooling to conv layers.

Parameters: 

- `ksize` size of the filter `[1,2,2,1]` is common
- `strides` `[1,2,2,1]` is common
- `padding` 

First and last dimensions in the ksize/stride array are for `batch` and `channels`, and are usually set to `1`. 

In [3]:
conv_layer = tf.nn.max_pool(
    conv_layer,
    ksize = [1,2,2,1],
    strides = [1,2,2,1],
    padding = 'SAME')

## Full Example CNN in TensorFlow 

### Dataset 

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters 
learning_rate = .1
epochs = 100
batch_size = 128

# Samples for validation and test sets 
test_valid_size = 256

# Network params
num_classes = 10
keep_prob = .75

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


### Weights and Biases

In [6]:
weights = {
    'conv1': tf.Variable(tf.random_normal([5,5,1,32])),
    'conv2': tf.Variable(tf.random_normal([5,5,32,64])),
    'fc1': tf.Variable(tf.random_normal([7*7*64,1024])),
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'conv1': tf.Variable(tf.random_normal([32])),
    'conv2': tf.Variable(tf.random_normal([64])),
    'fc1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

### Convolutions

![](image/convolution-schematic.gif)

Convolution w/ 3x3 filter and stride of 1 being applied to data.

The convolution for each 3x3 section is calculated against the weight, `[[1, 0, 1], [0, 1, 0], [1, 0, 1]]`, then a bias is added to create the convolved feature on the right. In this case, the bias is zero.

In [7]:
def conv2d(x, W, b, stride = 1, padding='SAME'):
    hidden = tf.nn.conv2d(x, W, strides=[1,stride,stride,1], padding=padding)
    hidden = tf.nn.bias_add(hidden, b)
    return tf.nn.relu(hidden)

### Max Pooling

![](image/maxpool.jpeg)

Max pooling with 2x2 filter and a stride of 2. The colors in put represent each time the filter was applied to create the max on the right. 

For example, `[[1, 1], [5, 6]]` becomes 6 and `[[3, 2], [1, 2]]` becomes 3.

In [8]:
def maxpool2d(conv, size=2):
    return tf.nn.max_pool(
        conv,
        ksize=[1,size,size,1],
        strides=[1,size,size,1],
        padding='SAME')

### Model

![](image/arch.png)

We're going to create 3 layers alternating b/w convs and max pooling followed by a fully connected and output layer. The transformation with the new dimensions are shown 